In [67]:
import plotly.express as px
from utils.ops import load_yaml
from  pathlib import Path
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [88]:
site = 2
site_label  = 'PA'

In [89]:
cfg = load_yaml('cfg.yaml')
site_cfg = load_yaml(f'site_{site}.yaml')

preparation_params = cfg['preparation_params']
paths_params = cfg['paths']
general_params = cfg['general_params']
experiments_paths = general_params['experiments_folders']

In [110]:
colors = [
    'aqua',
    'chocolate',
    'darkgreen',
    'brown',
    'cornflowerblue',
    'gold',
    'lightcoral',
    'cadetblue',
    'lightgreen',
    'cyan',
    'mediumpurple',
]

In [94]:
def results(experiments):
    exp_results = []
    for exp_i in experiments:
        #experiment_params = cfg['experiments'][f'exp_{experiments[exp_i]}']
        exp_path = Path(paths_params['experiments']) / f'exp_{exp_i}'
        experiment_params = cfg['experiments'][f'exp_{exp_i}']

        results_path = exp_path / experiments_paths['results']
        final_results_file = results_path / f'results_{exp_i}.data'

        result_df = pd.read_pickle(final_results_file)
        exp_results.append(
            {
                'name': experiment_params['short'],
                'Cloud Condition': 'Global',
                'metric': 'Precision',
                'value': result_df['global_precision'][0],
                'group_model': experiment_params['group_model']
            }
        )
        exp_results.append(
            {
                'name': experiment_params['short'],
                'Cloud Condition': 'Global',
                'metric': 'Recall',
                'value': result_df['global_recall'][0],
                'group_model': experiment_params['group_model']
            }
        )
        exp_results.append(
            {
                'name': experiment_params['short'],
                'Cloud Condition': 'Global',
                'metric': 'F1-Score',
                'value': result_df['global_f1'][0],
                'group_model': experiment_params['group_model']
            }
        )
        exp_results.append(
            {
                'name': experiment_params['short'],
                'Cloud Condition': 'No Cloud',
                'metric': 'Precision',
                'value': result_df['no_cloud_precision'][0],
                'group_model': experiment_params['group_model']
            }
        )
        exp_results.append(
            {
                'name': experiment_params['short'],
                'Cloud Condition': 'No Cloud',
                'metric': 'Recall',
                'value': result_df['no_cloud_recall'][0],
                'group_model': experiment_params['group_model']
            }
        )
        exp_results.append(
            {
                'name': experiment_params['short'],
                'Cloud Condition': 'No Cloud',
                'metric': 'F1-Score',
                'value': result_df['no_cloud_f1'][0],
                'group_model': experiment_params['group_model']
            }
        )
        exp_results.append(
            {
                'name': experiment_params['short'],
                'Cloud Condition': 'Cloudy',
                'metric': 'Precision',
                'value': result_df['cloud_precision'][0],
                'group_model': experiment_params['group_model']
            }
        )
        exp_results.append(
            {
                'name': experiment_params['short'],
                'Cloud Condition': 'Cloudy',
                'metric': 'Recall',
                'value': result_df['cloud_recall'][0],
                'group_model': experiment_params['group_model']
            }
        )
        exp_results.append(
            {
                'name': experiment_params['short'],
                'Cloud Condition': 'Cloudy',
                'metric': 'F1-Score',
                'value': result_df['cloud_f1'][0],
                'group_model': experiment_params['group_model']
            }
        )
        
        
    return pd.DataFrame(exp_results)


In [145]:
experiments = [
    6,
    1,
    2,
    3,
    4,
    5,
    7,
    8,
    10,
    26,
    21,
    22,
    23,
    24
]
pixel_cond = 'Global'
metric = 'F1-Score'
exp_results = results(experiments)
filt_res = exp_results[(exp_results['Cloud Condition'] == pixel_cond) & (exp_results['metric'] == metric)]
fig = make_subplots(rows=1, cols = 2, subplot_titles=('CNN', 'SWIN-Unet'), shared_yaxes = True)

cnn_results = filt_res[filt_res['group_model'] == 'CNN']
for i, name_res in enumerate(cnn_results['name']):
    loc_res = cnn_results[cnn_results['name'] == name_res]
    #print(loc_res)
    fig.add_bar( x = loc_res['name'], y=loc_res['value'], name = loc_res['name'].iloc[0], text=loc_res['value'], texttemplate='%{text:.2f}', textposition='outside', row=1, col=1, marker_color = px.colors.qualitative.Plotly[i])

swin_results = filt_res[filt_res['group_model'] == 'SWIN-Unet']
for i, name_res in enumerate(swin_results['name']):
    loc_res = swin_results[swin_results['name'] == name_res]
    #print(loc_res)
    fig.add_bar( x = loc_res['name'], y=loc_res['value'], name = loc_res['name'].iloc[0], showlegend = False, text=loc_res['value'], texttemplate='%{text:.2f}', textposition='outside', row=1, col=2, marker_color = px.colors.qualitative.Plotly[i])


fig.update_yaxes(range=[0.44, 1], title_text = '', tickformat = '.1f') 
fig.update_layout(
    width = 1600,
    height = 500,
    legend_title_text = 'Model', 
    legend_font_size = 16,
    #legend_orientation = 'h',
    font_size = 12,
    bargap=0.30,
    title_text = f'{metric} ({pixel_cond})',
    title_font_size = 20,
    title_x = 0.5
    )
fig.show()


In [70]:
experiments = {
    'Opt (Cloud-free)': 6,
    'Opt (Diverse Cond)': 1,
    'SAR (12 Avg Images)': 2,
    'Early Fusion (Diverse Opt + 12 Avg SAR)': 3,
    'Joint Fusion (Diverse Opt + 12 Avg SAR)': 4,
    'Late Fusion (Diverse Opt + 12 Avg SAR)': 5,
    'Joint Fusion - pretrained (Diverse Opt + 12 Avg SAR)': 7,
    'Late Fusion - pretrained (Diverse Opt + 12 Avg SAR)': 8,
    'Late Fusion - pretrained (Multitask Opt + 12 Avg SAR)': 10,
}
metric = 'F1-Score'
exp_results = results(experiments)
exp_results = exp_results.loc[exp_results['Cloud Condition'] == 'Global']
exp_results = exp_results.loc[~((exp_results['Cloud Condition'] != 'Global') & ( (exp_results['name'] == 'SAR (12 Avg Images)') | (exp_results['name'] == 'Opt (Cloud-free)') ) )]
exp_results = exp_results.loc[exp_results['metric'] == metric]
fig = px.bar(exp_results, x = 'Cloud Condition', y= 'value', color='name', barmode='group', text='value', facet_col = 'metric', width=1400, height=400)
fig.update_xaxes(categoryorder='array', categoryarray= ['No Cloud', 'Cloudy', 'Global'])
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_yaxes(range=[0, 1], title_text = '', tickformat = '.1f') 
fig.update_xaxes(
    title_text = '',
    tickfont_size = 18
    ) 
fig.update_layout(
    legend_title_text = 'Model', 
    legend_font_size = 16,
    legend_orientation = 'h',
    font_size = 12,
    bargap=0.30,
    title_text = f'Site: {site_label} ({pixel_cond})',
    title_x = 0.5
    )
#fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.for_each_annotation(lambda a: a.update(text=''))
fig.show()

KeyError: 'exp_Opt (Cloud-free)'

In [7]:
experiments = {
    'SAR (12 Avg Images)': 2,
    'SAR (2 Avg Images)': 7,
    'SAR (2 Single Images)': 8,
    'SAR (All Single Images)': 9,
}
exp_results = results(experiments)
exp_results = exp_results.loc[exp_results['Cloud Condition'] == 'Global']
#exp_results = exp_results.loc[~((exp_results['Cloud Condition'] != 'Global') & ( (exp_results['name'] == 'SAR (12 Avg Images)') | (exp_results['name'] == 'Opt (Cloud-free)') ) )]
fig = px.bar(exp_results, x = 'Cloud Condition', y= 'value', color='name', barmode='group', text='value', facet_col = 'metric', width=1400, height=400)
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_yaxes(range=[0, 1], title_text = '', tickformat = '.1f') 
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_xaxes(
    title_text = '',
    visible = False
    ) 
fig.update_layout(
    legend_title_text = 'Model', 
    legend_font_size = 16,
    font_size = 14,
    bargap=0.30,
    title_text = f'Site: {site_label}',
    title_x = 0.5
    )

#fig.for_each_annotation(lambda a: a.update(text=''))
fig.show()

In [8]:
experiments = {
    'Opt (WBCE, ignore class 2)': '6_wbce_2_8_adamw',
    'Opt (Focal gamma=3, ignore class 2)': '6_gamma3-lr-2e-5',
    'Opt (Focal gamma=5, ignore class 2)': '6_gamma5-lr-2e-5',
    'Opt (Focal gamma=3, include class 2)': '6_gamma3-comcls2',
}
exp_results = results(experiments)
exp_results = exp_results.loc[exp_results['Cloud Condition'] == 'Global']
#exp_results = exp_results.loc[~((exp_results['Cloud Condition'] != 'Global') & ( (exp_results['name'] == 'SAR (12 Avg Images)') | (exp_results['name'] == 'Opt (Cloud-free)') ) )]
fig = px.bar(exp_results, x = 'Cloud Condition', y= 'value', color='name', barmode='group', text='value', facet_col = 'metric', width=1400, height=400)
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_yaxes(range=[0, 1.05], title_text = '', tickformat = '.1f') 
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_xaxes(
    title_text = '',
    visible = False
    ) 
fig.update_layout(
    legend_title_text = 'Model', 
    legend_font_size = 16,
    font_size = 14,
    bargap=0.30,
    title_text = f'Site: {site_label}',
    title_x = 0.5
    ) 

#fig.for_each_annotation(lambda a: a.update(text=''))
fig.show()

FileNotFoundError: [Errno 2] No such file or directory: '/home/felferrari/projects/thesis/experiments/exp_6_wbce_2_8_adamw/results/results_6_wbce_2_8_adamw.data'

In [ ]:
experiments = {
    'Opt (w/out 3d Conv)': 6,
    'Opt (with 3d Conv)': 16,
}
exp_results = results(experiments)
exp_results = exp_results.loc[exp_results['Cloud Condition'] == 'Global']
#exp_results = exp_results.loc[~((exp_results['Cloud Condition'] != 'Global') & ( (exp_results['name'] == 'SAR (12 Avg Images)') | (exp_results['name'] == 'Opt (Cloud-free)') ) )]
fig = px.bar(exp_results, x = 'Cloud Condition', y= 'value', color='name', barmode='group', text='value', facet_col = 'metric', width=1400, height=400)
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_yaxes(range=[0, 1.05], title_text = '', tickformat = '.1f') 
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_xaxes(
    title_text = '',
    visible = False
    ) 
fig.update_layout(
    legend_title_text = 'Model', 
    legend_font_size = 16,
    font_size = 14,
    bargap=0.30,
    title_text = f'Site: {site_label}',
    title_x = 0.5
    ) 

#fig.for_each_annotation(lambda a: a.update(text=''))
fig.show()

In [7]:
experiments = {
    'CNN Opt (Diverse Cond)': 1,
    'CNN Opt (Cloud-free)': 6,
    'CNN SAR (12 Avg Images)': 2,
    #'CNN Early Fusion (Diverse Opt + 12 Avg SAR)': 3,
    #'CNN Joint Fusion (Diverse Opt + 12 Avg SAR)': 4,
    #'CNN Late Fusion (Diverse Opt + 12 Avg SAR)': 5,
    'CNN Joint Fusion - pretrained (Diverse Opt + 12 Avg SAR)': 11,
    'CNN Late Fusion - pretrained (Diverse Opt + 12 Avg SAR)': 12,
    'SWIN Opt (Diverse Cond)': 21,
    'SWIN Opt (Cloud-free)': 26,
    'SWIN SAR (12 Avg Images)': 22,
    #'SWIN Early Fusion (Diverse Opt + 12 Avg SAR)': 23,
    'SWIN Joint Fusion (Diverse Opt + 12 Avg SAR)': 24,
    'SWIN Late Fusion (Diverse Opt + 12 Avg SAR)': 25,
    #'Joint Fusion - pretrained (Diverse Opt + 12 Avg SAR)': 11,
    #'Late Fusion - pretrained (Diverse Opt + 12 Avg SAR)': 12,
}
pixel_cond = 'Global'
#pixel_cond = 'Cloudy'
#pixel_cond = 'No Cloud'
exp_results = results(experiments)
#exp_results = exp_results.loc[~((exp_results['Cloud Condition'] != 'Global') & ( (exp_results['name'] == 'SAR (12 Avg Images)') | (exp_results['name'] == 'Opt (Cloud-free)') ) )]
#exp_results = exp_results.loc[~((exp_results['Cloud Condition'] != 'Global') & ( (exp_results['name'] == 'CNN Opt (Cloud-free)') | (exp_results['name'] == 'SWIN Opt (Cloud-free)') ) )]
exp_results = exp_results.loc[exp_results['Cloud Condition'] == pixel_cond]
fig = px.bar(exp_results, x = 'metric', y= 'value', color='name', barmode='group', text='value', facet_col = 'Cloud Condition', width=1400, height=400)
#fig.update_xaxes(categoryorder='array', categoryarray= ['No Cloud', 'Cloudy', 'Global'])
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_yaxes(range=[0, 1.05], title_text = '', tickformat = '.1f') 
fig.update_xaxes(
    title_text = '',
    tickfont_size = 20
    ) 
fig.update_layout(
    legend_title_text = 'Model', 
    legend_font_size = 16,
    legend_orientation = 'h',
    font_size = 14,
    bargap=0.30,
    title_text = f'Site: {site_label} ({pixel_cond})',
    title_x = 0.5
    )
#fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.for_each_annotation(lambda a: a.update(text=''))
fig.show()

In [10]:
experiments = {
    'SWIN SAR (12 Avg Images)': 22,
    'SWIN SAR (2 Avg Images)': 27,
    'SWIN SAR (2 Single Images)': 28,
    'SWIN SAR (All Single Images)': 29,
}
exp_results = results(experiments)
exp_results = exp_results.loc[exp_results['Cloud Condition'] == 'Global']
#exp_results = exp_results.loc[~((exp_results['Cloud Condition'] != 'Global') & ( (exp_results['name'] == 'SAR (12 Avg Images)') | (exp_results['name'] == 'Opt (Cloud-free)') ) )]
fig = px.bar(exp_results, x = 'Cloud Condition', y= 'value', color='name', barmode='group', text='value', facet_col = 'metric', width=1400, height=400)
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_yaxes(range=[0, 1], title_text = '', tickformat = '.1f') 
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_xaxes(
    title_text = '',
    visible = False
    ) 
fig.update_layout(
    legend_title_text = 'Model', 
    legend_font_size = 16,
    font_size = 14,
    bargap=0.30,
    title_text = f'Site: {site_label}',
    title_x = 0.5
    )

#fig.for_each_annotation(lambda a: a.update(text=''))
fig.show()